### 数据清洗

In [ ]:
with open(r"D:\desktop\宏利 pdf 文件\纯文本\保單條款_2022_05(pdfplumber).txt","r",encoding="utf-8") as f:
    #lines = f.readlines()
    text = f.read()
if text.startswith('\ufeff'):##去除文件开头的 BOM
    # BOM (Byte Order Mark) 是 UTF-8 编码文件的一个特殊字符，表示文件的编码格式
    text = text[1:]

In [12]:
import re
import os
with open(r"D:\desktop\宏利 pdf 文件\纯文本\保單條款_2022_05(pdfplumber).txt", "r", encoding="utf-8") as f:
    text = f.read()
# 匹配“备注...”和紧跟其后的编号
pattern = r"""备注︰此简体中文本只供参考用途，并非保单之一部分，亦不构成对任何条款之诠译或赋予任何条款之涵义。\s*\d{1,2}-[A-Z]+(?:-[A-Z]+)* \(\d{2}/\d{4}\)"""
cleaned_text = re.sub(pattern, '', text)
cleaned_text = re.sub(r'\s*此页为空白页\s*', '', cleaned_text)
# 清理所有可能多余的空行
cleaned_text = re.sub(r'\n{3,}', '\n\n', cleaned_text)
save_folder = 'clean'
os.makedirs(save_folder, exist_ok=True)
with open("./clean/保單條款_2022_05.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)

In [2]:
import os

from loguru import logger
from magic_pdf.pipe.UNIPipe import UNIPipe
from magic_pdf.rw.DiskReaderWriter import DiskReaderWriter

pdf_name = '2024全球经济金融展望报告'
pdf_path = os.path.join('data', "D:\desktop\宏利 pdf 文件\纯文本\宏利環球貨幣保障計劃 保單條款_2022_05.pdf")
output_dir = os.path.join('data', pdf_name)
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
local_image_dir = os.path.join(output_dir, 'images')
output_filename = os.path.join(output_dir, pdf_name)

if os.path.exists(output_filename + ".md"):
    logger.info("The preprocessing result already exists.")

try:
    pdf_bytes = open(pdf_path, "rb").read()
    jso_useful_key = {"_pdf_type": "", "model_list": []}
    image_dir = str(os.path.basename(local_image_dir))
    image_writer = DiskReaderWriter(local_image_dir)
    pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
    pipe.pipe_classify()
    pipe.pipe_analyze()
    pipe.pipe_parse()
    md_content = pipe.pipe_mk_markdown(image_dir, drop_mode="none")
    with open(f"{output_filename}.md", "w", encoding="utf-8") as f:
        f.write(md_content)
except Exception as e:
    logger.error(e)

ModuleNotFoundError: No module named 'magic_pdf.pipe'